## 0. Import librariess

In [3]:
!git add .
!git commit -m "Testing to see if git works with new setup"
!git push origin main

[main bfd8c57] Testing to see if git works with new setup
 1 file changed, 30 insertions(+)
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 8 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 594.13 KiB | 11.00 MiB/s, done.
Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), done.
To https://github.com/paigea58/disasterNLP.git
   11b6208..bfd8c57  main -> main


In [19]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade
!pip install tensorflow-text

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text


In [14]:
import tensorflow as tf
import pandas as pd
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend


## 1. Data import

In [2]:
# Read in training data file
fileName = 'train.csv'
train = pd.read_csv('train.csv')

# Display contents of train set
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# Data structure
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [4]:
# Read in test data file
test = pd.read_csv('test.csv', delimiter=',')

# Display contents of test set
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
# Data structure
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


## 2. Data exploration

In [6]:
# Both training and test sets are similar in length

train["length"] = train["text"].apply(len)
print("Train text length statistics:")
print(train["length"].describe())
print()

test["length"] = test["text"].apply(len)
print("Test text length statistics:")
print(test["length"].describe())

print()
print("Instances of each class:")
print(train.target.value_counts())


Train text length statistics:
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64

Test text length statistics:
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64

Instances of each class:
target
0    4342
1    3271
Name: count, dtype: int64


In [7]:
batchSize = 32
trainingExamples = train.shape[0]
trainSplit = 0.8
valSplit = 0.2
stepsPerEpoch = int(trainingExamples) * trainSplit // batchSize

epochs = 2
auto = tf.data.experimental.AUTOTUNE

In [11]:
from sklearn.model_selection import train_test_split

X = train["text"]
y = train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = valSplit, random_state = 42)

X_test = test["text"]

In [18]:
preset = "distil_bert_base_en_uncased"

preprocess = keras_nlp.models.DistilBertPreprocessor.from_preset(preset, 
                                                                 sequence_length=160, 
                                                                 name = "preprocessor_4_tweets")

classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes = 2)
classifier.summary()

TypeError: <class 'keras_nlp.src.models.distil_bert.distil_bert_tokenizer.DistilBertTokenizer'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras_nlp.src.models.distil_bert.distil_bert_tokenizer', 'class_name': 'DistilBertTokenizer', 'config': {'name': 'distil_bert_tokenizer', 'trainable': True, 'dtype': 'int32', 'vocabulary': None, 'sequence_length': None, 'lowercase': True, 'strip_accents': False, 'split': True, 'suffix_indicator': '##', 'oov_token': '[UNK]'}, 'registered_name': 'keras_nlp>DistilBertTokenizer', 'assets': ['assets/tokenizer/vocabulary.txt'], 'weights': None}.

Exception encountered: Error when deserializing class 'DistilBertTokenizer' using config={'name': 'distil_bert_tokenizer', 'trainable': True, 'dtype': 'int32', 'vocabulary': None, 'sequence_length': None, 'lowercase': True,